In [ ]:
import tkinter as tk
from tkinter import *
import tkinter.ttk as ttk
import csv
import os
import pandas as pd
import numpy as np
import PIL
from PIL import Image, ImageTk
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import sys
import subprocess
import os

#Values for the Rexroth products
Desired_Predictions =[{'valve_number': 17010138, 'desired_lower': 17.991, 'desired_middle1': 17.993, 'desired_middle2': 17.998 ,'desired_higher': 18.001}, 
                      {'valve_number': 17010271, 'desired_lower': 17.991, 'desired_middle1': 17.993, 'desired_middle2': 17.998 ,'desired_higher': 18.001},
                      {'valve_number': 17010847, 'desired_lower': 18.145, 'desired_middle1': 18.147, 'desired_middle2': 18.153 ,'desired_higher': 18.155},
                      {'valve_number': 17008144, 'desired_lower': 18.145, 'desired_middle1': 18.147, 'desired_middle2': 18.153 ,'desired_higher': 18.155},
                      {'valve_number': 17010838, 'desired_lower': 18.145, 'desired_middle1': 18.147, 'desired_middle2': 18.153 ,'desired_higher': 18.155},
                      {'valve_number': 17010799, 'desired_lower': 18.145, 'desired_middle1': 18.147, 'desired_middle2': 18.153 ,'desired_higher': 18.155}]

Reference_Table = [{'Product_Image': "C:/Users/Rexroth/Nikita/Final_code_with_GUI/Automation_curr/SB23.PNG", 'Product_Name': 'SB23-EHS1', 'Product_ID': [17010138, ',',17010271], 'Nominal_Dimensions': '17.991 - 18.001'}, 
                   {'Product_Image': "C:/Users/Rexroth/Nikita/Final_code_with_GUI/Automation_curr/SB33.PNG", 'Product_Name': 'SB33-EHS1 RD', 'Product_ID': [17010847,',', 17008144], 'Nominal_Dimensions': '18.15 +/- 0.005'},
                   {'Product_Image': "C:/Users/Rexroth/Nikita/Final_code_with_GUI/Automation_curr/EHR33.PNG", 'Product_Name': 'EHR33-EHS1', 'Product_ID': [17010838,',', 17010799], 'Nominal_Dimensions': '18.15 +/- 0.005'}]

#Refreshing the prediction table on updation of csv file
def frame_refresh(frame1,tree):
    file_Name = r"D:\Predictions\MongoData\Predictions.csv"

    reader_prev = pd.read_csv(file_Name , encoding = 'ISO-8859-1', engine='python')
    reader_prev= reader_prev.T[1:5].T
    reader_prev= reader_prev.iloc[::-1].reset_index(drop=True) #Read from last value of the csv file
    
    #Clearing all values first and then updating new values
    if len(tree.get_children()) > 0:
     for i in tree.get_children():
      tree.delete(i)

    for row in range(0,len(reader_prev)):
      if row <= (len(reader_prev)-1):     
       WorkpieceNumber = reader_prev['WorkpieceNumber'][row];
       Prediction = reader_prev['Predictions'][row];

       for col in range(0, len(Desired_Predictions)):
         if Desired_Predictions[col]['valve_number'] == int(WorkpieceNumber.split("_")[0]):
          if float(Prediction) <= Desired_Predictions[col]['desired_lower']:
           tag = 'red';
          elif float(Prediction) > Desired_Predictions[col]['desired_lower'] and float(Prediction) < Desired_Predictions[col]['desired_middle1']:
           tag = 'orange';
          elif float(Prediction) >= Desired_Predictions[col]['desired_middle1'] and float(Prediction) <= Desired_Predictions[col]['desired_middle2']:
           tag = 'green';
          elif float(Prediction) > Desired_Predictions[col]['desired_middle2'] and float(Prediction) < Desired_Predictions[col]['desired_higher']:
           tag = 'orange';
          elif float(Prediction) > Desired_Predictions[col]['desired_higher']:
           tag = 'red';
          else:
           tag = 'black';

          tree.insert("", row, values=(WorkpieceNumber, Prediction), tags=(tag));    
    ttk.Style().configure('Treeview', rowheight=30, font=("Helvetica", 12));

    tree.tag_configure('orange', foreground='orange')
    tree.tag_configure('red', foreground='red')
    tree.tag_configure('green', foreground='green')  
    tree.tag_configure('black', foreground='black') 
    frame1.after(120000, lambda: frame_refresh(frame1, tree)) #Refreshing UI every 2mins

#Creating the structure of the GUI
def prediction_GUI():    
    root = Tk() #Creating a window
    style = ttk.Style()
    root.title("Diameter: Predicted & Desired")

    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    width = 400;
    height = 60;
    root.geometry("%dx%d+0+0" % (screen_width, screen_height)) #Window Height
    m = root.maxsize()

    #Creating FRAME1(PREDICTION TABLE FRAME)
    frame1=LabelFrame(root, text='DIAMETER PREDITION FOR DIFFERENT VALVE NUMBERS', padx=0, pady=5, 
               relief=RAISED, font=("Helvetica", 9, 'bold')); #Frame1 inside root(Window)

    frame1.pack(padx=20,pady=20,side=LEFT);

    #Creating the table
    TableMargin = Frame(frame1, width=width, height=height)
    TableMargin = Frame(frame1);
    TableMargin.pack(side=LEFT)
    scrollbarx = Scrollbar(TableMargin, orient=HORIZONTAL)
    scrollbary = Scrollbar(TableMargin, orient=VERTICAL)
    tree = ttk.Treeview(TableMargin, columns=("WorkpieceNumber", "Predictions"), 
                height=height, selectmode="none", yscrollcommand=scrollbary.set, xscrollcommand=scrollbarx.set,
                style=('Treeview'))
    scrollbary.config(command=tree.yview)
    scrollbary.pack(side=RIGHT, fill=Y)
    scrollbarx.config(command=tree.xview)
    scrollbarx.pack(side=BOTTOM, fill=X)
    tree.heading('WorkpieceNumber', text="Workpiece Number")
    tree.heading('Predictions', text="Prediction")
    tree.column('#0', stretch=NO, minwidth=0, width=0, anchor=tk.CENTER )
    tree.column('#1', stretch=NO, minwidth=0, width=260, anchor=tk.CENTER)
    tree.column('#2', stretch=NO, minwidth=0, width=180, anchor=tk.CENTER)
    tree.pack(); 
    
    #Creating within FRAME1(COLOR REFERENCE)
    frame4 = Frame(frame1, relief=RAISED); #Frame2 inside root(Window)
    canvas = tk.Canvas(frame4, width = 160, height = 60, background='#ebebeb', highlightbackground="gray");
    canvas.create_text(80,10, fill="red",font=("Helvetica", 9, 'bold') ,text="Toleranz überschritten")
    canvas.create_text(80,30, fill="green",font=("Helvetica", 9, 'bold') ,text="Innerhalb Toleranz")
    canvas.create_text(80,50, fill="orange",font=("Helvetica", 9, 'bold') ,text="Nähe Toleranzgrenze")
    canvas.pack(side=RIGHT)
    frame4.pack(side=TOP)

    ##################################################################################################################################
    #Creating FRAME2(REFERENCE BOX)
    frame2 = LabelFrame(root, text='REFERENCE VALUES FOR DIAMETER',  
                        padx=20, pady=5, font=("Helvetica", 9, 'bold')); #Frame2 inside root(Window)
    frame2.pack(padx=5,pady=20,side=LEFT)

    #Creating the table
    Table = Frame(frame2, width=width, height=height)
    Table = Frame(frame2)
    Table.pack(side=LEFT)
    scrollbarx = Scrollbar(Table, orient=HORIZONTAL)
    scrollbary = Scrollbar(Table, orient=VERTICAL)
    tree1 = ttk.Treeview(Table, columns=("Product_Name", "Product_ID", "Nominal_Dimensions"),
                          height=height, selectmode="none",  yscrollcommand=scrollbary.set, xscrollcommand=scrollbarx.set, style=('MyStyle.Treeview'))
    scrollbary.config(command=tree1.yview)
    scrollbary.pack(side=RIGHT)
    scrollbarx.config(command=tree1.xview)
    scrollbarx.pack(side=BOTTOM, fill=X)
    tree1.heading('Product_Name', text="Product Name" , image= '')
    tree1.heading('Product_ID', text="Product ID")
    tree1.heading('Nominal_Dimensions', text="Nominal Dimensions")
    tree1.column('#0', stretch=YES, minwidth=0, width=200, anchor=tk.CENTER)
    tree1.column('#1', stretch=NO, minwidth=0, width=250, anchor=tk.CENTER)
    tree1.column('#2', stretch=NO, minwidth=0, width=250, anchor=tk.CENTER)
    tree1.column('#3', stretch=NO, minwidth=0, width=250, anchor=tk.CENTER)
    tree1.pack(); 
    
    reader2 = pd.DataFrame(Reference_Table)

    tree1._images = [];
    for row in range(0,len(reader2)):
      if row <= (len(reader2)-1):
       ProdImg = reader2['Product_Image'][row];
       ProdName = reader2['Product_Name'][row];
       ProdID = reader2['Product_ID'][row];
       ProdNominal = reader2['Nominal_Dimensions'][row];

       image = PIL.Image.open(ProdImg)
       image = PIL.ImageTk.PhotoImage(image, master=frame2)
       tree1._images.append(image)
       tree1.insert('', row, image=tree1._images[-1], values=(ProdName,ProdID,ProdNominal) )
       ttk.Style().configure('MyStyle.Treeview', rowheight=200, font=("Helvetica", 11, 'bold'), background='gray', foreground='white')

    #Rexroth Logo in FRAME3
    frame3= Frame(frame2, relief=RAISED); #Frame2 inside root(Window)
    Rexrothimage = Image.open('C:/Users/Rexroth/Nikita/Final_code_with_GUI/Automation_curr/Rexroth_Logo.png')
    canvas = tk.Canvas(frame3, width = 155, height = 100);
    logo = PIL.ImageTk.PhotoImage(Rexrothimage, master = canvas)
    canvas.create_image(80, 50, image = logo)
    canvas.image = logo;
    canvas.pack(side=RIGHT)
    frame3.pack(side=BOTTOM)

    frame_refresh(frame1, tree); #Second function call
    root.mainloop();        
    
prediction_GUI() #First function call  